The goal here is to define some functions to find interface residues (only on the peptide side), choose a specified number, and mutate them.

In [1]:
from pyrosetta import *
init()

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.CentOS.python37.Release 2020.10+release.46415fa3e9decb8b6e91a4e065c15543eb27a461 2020-03-05T09:09:24] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.CentOS.python37.Release r248 2020.10+release.46415fa 46415fa3e9decb8b6e91a4e065c15543eb27a461 http://www.pyrosetta.org 2020-03-05T09:09:24
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /home/colin/anaconda3/envs/pyrosetta/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=182823386 seed_offset=0 real_seed=182823386 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=182823386 RG_type=mt19937


In [1]:
%%writefile MutateInterface.py

# put it all together!!

import pyrosetta.rosetta.protocols as protocols
import random
    
def interface_res(pose, interface_distance=8, exclude_chain_B=True):
    """
    Returns a list of ints representing interface residue positions.
    """
    interface = protocols.interface.select_interface_residues(pose, 'A_B', interface_distance)
    
    if exclude_chain_B:
        B_chain_start = pose.chain_end(1)
        for i, res in enumerate(interface):
            if i > B_chain_start:
                interface[i] = 0
                
    # convert to a list of residue numbers
    interface_res = []
    for i, res in enumerate(interface):
        if res == 1:
            interface_res.append(i+1)
    
    return interface_res
#    return random.choice(interface_res)

def choose_AA(aas=["A", "V", "I", "L", "L", "F", "Y", "W", "S", "T", "N", "Q", "G", "R", "H", "K", "D", "E"]):
    return random.choice(aas)

def point_mutation_and_dock(pose, ndecoys):
    position = random.choice(interface_res(pose))
    aa = choose_AA()
    toolbox.mutants.mutate_residue(pose, position, aa, pack_radius=8)
    
    pep_run("./decoys/"+str(position)+aa, ndecoys, pose)
    
def point_mutate(pose, position, aa, pack_radius=8):
    toolbox.mutants.mutate_residue(pose, position, aa, pack_radius=8)

Overwriting MutateInterface.py


Looks like that works. Now try it with ```mutate_residue```

In [25]:
%%time
mut = Pose()
mut.assign(nbit)

position = choose_interface_res(mut)
aa = choose_AA()
toolbox.mutants.mutate_residue(mut, position, aa, pack_radius=8)
print("Position: {}".format(position))
print("AA: {}".format(aa))

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 412 rotamers at 15 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
Position: 22
AA: L
CPU times: user 161 ms, sys: 0 ns, total: 161 ms
Wall time: 159 ms


In [ ]:
from multiprocessing import Pool
import pyrosetta.rosetta.protocols as protocols

with Pool() as p:
    work = [(pose_from_pdb("./smbits/peptide114.pdb"), 50) for n in range(50)]
    p.starmap(point_mutation_and_dock, work)

core.import_pose.import_pose: {0} File './smbits/peptide114.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD  on residue GLU 24
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE1 on residue GLU 24
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE2 on residue GLU 24
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD  on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OE1 on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  NE2 on residue GLN 73
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CE  on residue LYS 76
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  NZ  on residue LYS 76
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue

In [50]:
from pyrosetta import PyMOLMover

pmm = PyMOLMover("192.168.50.75", 65000) #enter the IP that pymol displays on startup
pmm.keep_history(True)
pmm.apply(mut)